In [ ]:
%matplotlib inline


How to reproduce the doc2vec 'Paragraph Vector' paper
=====================================================

Shows how to reproduce results of the "Distributed Representation of Sentences and Documents" paper by Le and Mikolov using Gensim.




In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Introduction
------------

This guide shows you how to reproduce the results of the paper by `Le and
Mikolov 2014 <https://arxiv.org/pdf/1405.4053.pdf>`_ using Gensim. While the
entire paper is worth reading (it's only 9 pages), we will be focusing on
Section 3.2: "Beyond One Sentence - Sentiment Analysis with the IMDB
dataset".

This guide follows the following steps:

#. Load the IMDB dataset
#. Train a variety of Doc2Vec models on the dataset
#. Evaluate the performance of each model using a logistic regression
#. Examine some of the results directly:

When examining results, we will look for answers for the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?

Load corpus
-----------

Our data for the tutorial will be the `IMDB archive
<http://ai.stanford.edu/~amaas/data/sentiment/>`_.
If you're not familiar with this dataset, then here's a brief intro: it
contains several thousand movie reviews.

Each review is a single line of text containing multiple sentences, for example:

```
One of the best movie-dramas I have ever seen. We do a lot of acting in the
church and this is one that can be used as a resource that highlights all the
good things that actors can do in their work. I highly recommend this one,
especially for those who have an interest in acting, as a "must see."
```

These reviews will be the **documents** that we will work with in this tutorial.
There are 100 thousand reviews in total.

#. 25k reviews for training (12.5k positive, 12.5k negative)
#. 25k reviews for testing (12.5k positive, 12.5k negative)
#. 50k unlabeled reviews

Out of 100k reviews, 50k have a label: either positive (the reviewer liked
the movie) or negative.
The remaining 50k are unlabeled.

Our first task will be to prepare the dataset.

More specifically, we will:

#. Download the tar.gz file (it's only 84MB, so this shouldn't take too long)
#. Unpack it and extract each movie review
#. Split the reviews into training and test datasets

First, let's define a convenient datatype for holding data for a single document:

* words: The text of the document, as a ``list`` of words.
* tags: Used to keep the index of the document in the entire dataset.
* split: one of ``train``\ , ``test`` or ``extra``. Determines how the document will be used (for training, testing, etc).
* sentiment: either 1 (positive), 0 (negative) or None (unlabeled document).

This data type is helpful for later evaluation and reporting.
In particular, the ``index`` member will help us quickly and easily retrieve the vectors for a document from a model.




In [3]:
import collections

SentimentDocument = collections.namedtuple('SentimentDocument', 'words tags split sentiment')

We can now proceed with loading the corpus.



In [5]:
import io
import re
import tarfile
import os.path

import smart_open
import gensim.utils

def download_dataset(url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'):
    fname = url.split('/')[-1]

    if os.path.isfile(fname):
       return fname

    # Download the file to local storage first.
    with smart_open.open(url, "rb") as fin:
        with smart_open.open(fname, 'wb') as fout:
            while True:
                buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                if not buf:
                    break
                fout.write(buf)

    return fname

def create_sentiment_document(name, text, index):
    _, split, sentiment_str, _ = name.split('/')
    sentiment = {'pos': 1.0, 'neg': 0.0, 'unsup': None}[sentiment_str]

    if sentiment is None:
        split = 'extra'

    tokens = gensim.utils.to_unicode(text).split()
    return SentimentDocument(tokens, [index], split, sentiment)

def extract_documents():
    fname = download_dataset()

    index = 0

    with tarfile.open(fname, mode='r:gz') as tar:
        for member in tar.getmembers():
            if re.match(r'aclImdb/(train|test)/(pos|neg|unsup)/\d+_\d+.txt$', member.name):
                member_bytes = tar.extractfile(member).read()
                member_text = member_bytes.decode('utf-8', errors='replace')
                assert member_text.count('\n') == 0
                yield create_sentiment_document(member.name, member_text, index)
                index += 1

alldocs = list(extract_documents())

Here's what a single document looks like.



In [36]:
print(alldocs[24000])

SentimentDocument(words=['The', 'movie', 'is', 'about', 'a', 'girl', "who's", 'not', 'going', 'to', 'a', 'bonfire', 'only', 'because', "she's", 'baby-sitting', 'that', 'night.', 'Nothing', 'weird', 'about', 'that,', 'right?', 'Until', '...', 'The', 'phone', 'rings.', 'Until', '...', 'The', 'phone', 'rings', 'again.', 'And', 'again', '...', 'And', 'again.', 'Those', 'are', 'not', 'some', 'stupid', 'prank', 'calls.', 'This', 'is', 'for', 'real.', 'If', 'you', 'wanna', 'see', 'how', 'the', 'girl', 'reacts,', 'just', 'watch', 'the', 'movie.<br', '/><br', '/>Great', 'atmosphere', 'filled', 'with', 'scary', 'sounds.', 'Very', 'well', 'performed', 'by', 'young', 'Camilla', 'Belle', 'who', 'got', 'the', 'lead', 'role.', 'I', 'see', 'in', 'her', 'some', 'great', 'potential', 'to', 'become', 'a', 'good', 'actress.', 'This', 'is', 'more', 'than', 'only', 'a', 'decent', 'thriller,', 'I', 'have', 'no', 'idea', 'why', "it's", 'so', 'underrated.', 'Anyway,', 'on', 'my', 'opinion', 'this', 'movie', 'd

Extract our documents and split into training/test sets.



In [7]:
train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
print(f'{len(alldocs)} docs: {len(train_docs)} train-sentiment, {len(test_docs)} test-sentiment')

100000 docs: 25000 train-sentiment, 25000 test-sentiment


Set-up Doc2Vec Training & Evaluation Models
-------------------------------------------

We approximate the experiment of Le & Mikolov `"Distributed Representations
of Sentences and Documents"
<http://cs.stanford.edu/~quocle/paragraph_vector.pdf>`_ with guidance from
Mikolov's `example go.sh
<https://groups.google.com/d/msg/word2vec-toolkit/Q49FIrNOQRo/J6KG8mUj45sJ>`_::

    ./word2vec -train ../alldata-id.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 0 -sample 1e-4 -threads 40 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1

We vary the following parameter choices:

* 100-dimensional vectors, as the 400-d vectors of the paper take a lot of
  memory and, in our tests of this task, don't seem to offer much benefit
* Similarly, frequent word subsampling seems to decrease sentiment-prediction
  accuracy, so it's left out
* ``cbow=0`` means skip-gram which is equivalent to the paper's 'PV-DBOW'
  mode, matched in gensim with ``dm=0``
* Added to that DBOW model are two DM models, one which averages context
  vectors (\ ``dm_mean``\ ) and one which concatenates them (\ ``dm_concat``\ ,
  resulting in a much larger, slower, more data-hungry model)
* A ``min_count=2`` saves quite a bit of model memory, discarding only words
  that appear in a single doc (and are thus no more expressive than the
  unique-to-each doc vectors themselves)




In [8]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

common_kwargs = dict(
    vector_size=100, epochs=20, min_count=2,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print(f"{model} vocabulary scanned & state initialized")

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2024-05-07 16:12:10,938 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow,d100,n5,mc2,t8>', 'datetime': '2024-05-07T16:12:10.938995', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'created'}
2024-05-07 16:12:10,938 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d100,n5,w10,mc2,t8>', 'datetime': '2024-05-07T16:12:10.938995', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'created'}
2024-05-07 16:12:10,938 : INFO : using concatenative 1100-dimensional layer1
2024-05-07 16:12:10,938 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/c,d100,n5,w5,mc2,t8>', 'datetime': '2024-05-07T16:12:10.938995', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.

Doc2Vec<dbow,d100,n5,mc2,t8> vocabulary scanned & state initialized


2024-05-07 16:12:19,016 : INFO : PROGRESS: at example #10000, processed 2292381 words (5675641 words/s), 150816 word types, 0 tags
2024-05-07 16:12:19,376 : INFO : PROGRESS: at example #20000, processed 4573645 words (6262479 words/s), 238497 word types, 0 tags
2024-05-07 16:12:19,741 : INFO : PROGRESS: at example #30000, processed 6865575 words (6386203 words/s), 312348 word types, 0 tags
2024-05-07 16:12:20,132 : INFO : PROGRESS: at example #40000, processed 9190019 words (5897541 words/s), 377231 word types, 0 tags
2024-05-07 16:12:20,509 : INFO : PROGRESS: at example #50000, processed 11557847 words (6275154 words/s), 438729 word types, 0 tags
2024-05-07 16:12:20,909 : INFO : PROGRESS: at example #60000, processed 13899883 words (5832528 words/s), 493913 word types, 0 tags
2024-05-07 16:12:21,326 : INFO : PROGRESS: at example #70000, processed 16270094 words (5738096 words/s), 548474 word types, 0 tags
2024-05-07 16:12:21,747 : INFO : PROGRESS: at example #80000, processed 18598876

Doc2Vec<dm/m,d100,n5,w10,mc2,t8> vocabulary scanned & state initialized


2024-05-07 16:12:26,491 : INFO : PROGRESS: at example #10000, processed 2292381 words (6682562 words/s), 150816 word types, 0 tags
2024-05-07 16:12:26,841 : INFO : PROGRESS: at example #20000, processed 4573645 words (6522656 words/s), 238497 word types, 0 tags
2024-05-07 16:12:27,221 : INFO : PROGRESS: at example #30000, processed 6865575 words (5967291 words/s), 312348 word types, 0 tags
2024-05-07 16:12:27,641 : INFO : PROGRESS: at example #40000, processed 9190019 words (5551461 words/s), 377231 word types, 0 tags
2024-05-07 16:12:28,052 : INFO : PROGRESS: at example #50000, processed 11557847 words (5868233 words/s), 438729 word types, 0 tags
2024-05-07 16:12:28,437 : INFO : PROGRESS: at example #60000, processed 13899883 words (6050687 words/s), 493913 word types, 0 tags
2024-05-07 16:12:28,813 : INFO : PROGRESS: at example #70000, processed 16270094 words (6233201 words/s), 548474 word types, 0 tags
2024-05-07 16:12:29,185 : INFO : PROGRESS: at example #80000, processed 18598876

Doc2Vec<dm/c,d100,n5,w5,mc2,t8> vocabulary scanned & state initialized


Le and Mikolov note that combining a paragraph vector from Distributed Bag of
Words (DBOW) and Distributed Memory (DM) improves performance. We will
follow, pairing the models together for evaluation. Here, we concatenate the
paragraph vectors obtained from each model with the help of a thin wrapper
class included in a gensim test module. (Note that this a separate, later
concatenation of output-vectors than the kind of input-window-concatenation
enabled by the ``dm_concat=1`` mode above.)




In [10]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

2024-05-07 16:15:21,149 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-05-07 16:15:21,149 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2024-05-07 16:15:21,150 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2024-05-07T16:15:21.150095', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'created'}


Predictive Evaluation Methods
-----------------------------

Given a document, our ``Doc2Vec`` models output a vector representation of the document.
How useful is a particular model?
In case of sentiment analysis, we want the ouput vector to reflect the sentiment in the input document.
So, in vector space, positive documents should be distant from negative documents.

We train a logistic regression from the training set:

  - regressors (inputs): document vectors from the Doc2Vec model
  - target (outpus): sentiment labels

So, this logistic regression will be able to predict sentiment given a document vector.

Next, we test our logistic regression on the test set, and measure the rate of errors (incorrect predictions).
If the document vectors from the Doc2Vec model reflect the actual sentiment well, the error rate will be low.

Therefore, the error rate of the logistic regression is indication of *how well* the given Doc2Vec model represents documents as vectors.
We can then compare different ``Doc2Vec`` models by looking at their error rates.




In [12]:
import numpy as np
import statsmodels.api as sm
from random import sample

def logistic_predictor_from_data(train_targets, train_regressors):
    """Fit a statsmodel logistic predictor on supplied data"""
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.dv[doc.tags[0]] for doc in train_set]
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.dv[doc.tags[0]] for doc in test_set]
    test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_set])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

Bulk Training & Per-Model Evaluation
------------------------------------

Note that doc-vector training is occurring on *all* documents of the dataset,
which includes all TRAIN/TEST/DEV docs.  Because the native document-order
has similar-sentiment documents in large clumps – which is suboptimal for
training – we work with once-shuffled copy of the training set.

We evaluate each model's sentiment predictive power based on error rate, and
the evaluation is done for each model.

(On a 4-core 2.6Ghz Intel Core i7, these 20 passes training and evaluating 3
main models takes about an hour.)




In [13]:
from collections import defaultdict
error_rates = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [14]:
from random import shuffle
shuffled_alldocs = alldocs[:]
shuffle(shuffled_alldocs)

for model in simple_models:
    print(f"Training {model}")
    model.train(shuffled_alldocs, total_examples=len(shuffled_alldocs), epochs=model.epochs)

    print(f"\nEvaluating {model}")
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print("\n%f %s\n" % (err_rate, model))

for model in [models_by_name['dbow+dmm'], models_by_name['dbow+dmc']]:
    print(f"\nEvaluating {model}")
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print(f"\n{err_rate} {model}\n")

2024-05-07 16:16:43,474 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 265408 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=5 window=5 shrink_windows=True', 'datetime': '2024-05-07T16:16:43.474581', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'train'}


Training Doc2Vec<dbow,d100,n5,mc2,t8>


2024-05-07 16:16:44,487 : INFO : EPOCH 0 - PROGRESS: at 7.30% examples, 1682246 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:45,495 : INFO : EPOCH 0 - PROGRESS: at 15.97% examples, 1815587 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:46,497 : INFO : EPOCH 0 - PROGRESS: at 25.64% examples, 1951194 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:47,498 : INFO : EPOCH 0 - PROGRESS: at 36.98% examples, 2108581 words/s, in_qsize 16, out_qsize 0
2024-05-07 16:16:48,499 : INFO : EPOCH 0 - PROGRESS: at 47.71% examples, 2178615 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:49,499 : INFO : EPOCH 0 - PROGRESS: at 58.02% examples, 2209534 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:50,494 : INFO : EPOCH 0 - PROGRESS: at 68.36% examples, 2231003 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:51,497 : INFO : EPOCH 0 - PROGRESS: at 77.53% examples, 2213648 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:16:52,500 : INFO : EPOCH 0 - PROGRESS: at 87.78% examples, 2233104 


Evaluating Doc2Vec<dbow,d100,n5,mc2,t8>


2024-05-07 16:19:39,851 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 265408 vocabulary and 100 features, using sg=0 hs=0 sample=0 negative=5 window=10 shrink_windows=True', 'datetime': '2024-05-07T16:19:39.851558', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'train'}


...


Evaluating Doc2Vec<dm/m,d100,n5,w10,mc2,t8>


2024-05-07 16:24:26,890 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 8 workers on 265409 vocabulary and 1100 features, using sg=0 hs=0 sample=0 negative=5 window=5 shrink_windows=True', 'datetime': '2024-05-07T16:24:26.890395', 'gensim': '4.3.2', 'python': '3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22631-SP0', 'event': 'train'}



0.169760 Doc2Vec<dm/m,d100,n5,w10,mc2,t8>

Training Doc2Vec<dm/c,d100,n5,w5,mc2,t8>


2024-05-07 16:24:27,908 : INFO : EPOCH 0 - PROGRESS: at 2.25% examples, 518185 words/s, in_qsize 16, out_qsize 0
2024-05-07 16:24:28,908 : INFO : EPOCH 0 - PROGRESS: at 5.08% examples, 587939 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:24:29,912 : INFO : EPOCH 0 - PROGRESS: at 7.96% examples, 610336 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:24:30,909 : INFO : EPOCH 0 - PROGRESS: at 10.95% examples, 626375 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:24:31,956 : INFO : EPOCH 0 - PROGRESS: at 13.98% examples, 635672 words/s, in_qsize 16, out_qsize 0
2024-05-07 16:24:32,975 : INFO : EPOCH 0 - PROGRESS: at 17.07% examples, 643118 words/s, in_qsize 15, out_qsize 0
2024-05-07 16:24:33,983 : INFO : EPOCH 0 - PROGRESS: at 20.02% examples, 647957 words/s, in_qsize 16, out_qsize 0
2024-05-07 16:24:34,983 : INFO : EPOCH 0 - PROGRESS: at 22.94% examples, 650461 words/s, in_qsize 16, out_qsize 0
2024-05-07 16:24:35,992 : INFO : EPOCH 0 - PROGRESS: at 25.94% examples, 654862 words/s, in


Evaluating Doc2Vec<dm/c,d100,n5,w5,mc2,t8>

0.299760 Doc2Vec<dm/c,d100,n5,w5,mc2,t8>


Evaluating Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/m,d100,n5,w10,mc2,t8>

0.10328 Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/m,d100,n5,w10,mc2,t8>


Evaluating Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/c,d100,n5,w5,mc2,t8>

0.10552 Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/c,d100,n5,w5,mc2,t8>



Achieved Sentiment-Prediction Accuracy
--------------------------------------
Compare error rates achieved, best-to-worst



In [15]:
print("Err_rate Model")
for rate, name in sorted((rate, name) for name, rate in error_rates.items()):
    print(f"{rate} {name}")

Err_rate Model
0.10328 Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/m,d100,n5,w10,mc2,t8>
0.10408 Doc2Vec<dbow,d100,n5,mc2,t8>
0.10552 Doc2Vec<dbow,d100,n5,mc2,t8>+Doc2Vec<dm/c,d100,n5,w5,mc2,t8>
0.16976 Doc2Vec<dm/m,d100,n5,w10,mc2,t8>
0.29976 Doc2Vec<dm/c,d100,n5,w5,mc2,t8>


In our testing, contrary to the results of the paper, on this problem,
PV-DBOW alone performs as good as anything else. Concatenating vectors from
different models only sometimes offers a tiny predictive improvement – and
stays generally close to the best-performing solo model included.

The best results achieved here are just around 10% error rate, still a long
way from the paper's reported 7.42% error rate.

(Other trials not shown, with larger vectors and other changes, also don't
come close to the paper's reported value. Others around the net have reported
a similar inability to reproduce the paper's best numbers. The PV-DM/C mode
improves a bit with many more training epochs – but doesn't reach parity with
PV-DBOW.)




Examining Results
-----------------

Let's look for answers to the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?




Are inferred vectors close to the precalculated ones?
-----------------------------------------------------



In [17]:
doc_id = np.random.randint(len(simple_models[0].dv))  # Pick random doc; re-run cell for more examples
print(f'for doc {doc_id}...')
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print(f'{model}:\n {model.dv.most_similar([inferred_docvec], topn=3)}')

for doc 40253...
Doc2Vec<dbow,d100,n5,mc2,t8>:
 [(40253, 0.9738572835922241), (44693, 0.6579931378364563), (42498, 0.6414970755577087)]
Doc2Vec<dm/m,d100,n5,w10,mc2,t8>:
 [(40253, 0.9069275259971619), (53282, 0.665367066860199), (43539, 0.663693368434906)]
Doc2Vec<dm/c,d100,n5,w5,mc2,t8>:
 [(40253, 0.6365260481834412), (3844, 0.502201497554779), (57826, 0.4490082859992981)]


(Yes, here the stored vector from 20 epochs of training is usually one of the
closest to a freshly-inferred vector for the same words. Defaults for
inference may benefit from tuning for each dataset or model parameters.)




Do close documents seem more related than distant ones?
-------------------------------------------------------



In [19]:
import random

doc_id = np.random.randint(len(simple_models[0].dv))  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.dv.most_similar(doc_id, topn=len(model.dv))  # get *all* similar documents
print(f'TARGET ({doc_id}): «{" ".join(alldocs[doc_id].words)}»\n')
print(f'SIMILAR/DISSIMILAR DOCS PER MODEL {model}%s:\n')
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    s = sims[index]
    i = sims[index][0]
    words = ' '.join(alldocs[i].words)
    print(f'{label} {s}: «{words}»\n')

TARGET (98042): «I am really amazed how bad acting can really be, I guess that is an art too... You could not make it worse with the old form of Russian synchronization in movies, where every role is spoken by one (male) person and every sentence begins with "he sais" or "she sais"... Or wait a minute, is it maybe a new form of comedy? Is there a secret award for the worst movies - this would certainly make it to the top. I still really have no clue what it is about, a bomb attack on what by who and what for and... ugh, what is the matter with all the dialogs in the movie? Even first graders have a better vocabulary. But one has to admit, it is really hard to forget such a movie and you couldn't stop watching it either because you just can't believe what you see.»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dbow,d100,n5,mc2,t8>%s:

MOST (65619, 0.6676626205444336): «I can't believe that this is supposed to be made after the early 70's... I mean even a twelve year old with his home PC co

Somewhat, in terms of reviewer tone, movie genre, etc... the MOST
cosine-similar docs usually seem more like the TARGET than the MEDIAN or
LEAST... especially if the MOST has a cosine-similarity > 0.5. Re-run the
cell to try another random target document.




Do the word vectors show useful similarities?
---------------------------------------------




In [20]:
import random

word_models = simple_models[:]

def pick_random_word(model, threshold=10):
    # pick a random word with a suitable number of occurences
    while True:
        word = random.choice(model.wv.index_to_key)
        if model.wv.get_vecattr(word, "count") > threshold:
            return word

target_word = pick_random_word(word_models[0])
# or uncomment below line, to just pick a word from the relevant domain:
# target_word = 'comedy/drama'

for model in word_models:
    print(f'target_word: {repr(target_word)} model: {model} similar words:')
    for i, (word, sim) in enumerate(model.wv.most_similar(target_word, topn=10), 1):
        print(f'    {i}. {sim:.2f} {repr(word)}')
    print()

target_word: '3".' model: Doc2Vec<dbow,d100,n5,mc2,t8> similar words:
    1. 0.46 'coin?'
    2. 0.44 'raps.'
    3. 0.42 '"uprising"'
    4. 0.41 'interest?'
    5. 0.41 'Goat,'
    6. 0.41 'locating'
    7. 0.40 'Romero.'
    8. 0.40 'Heart.'
    9. 0.40 '"Disco'
    10. 0.40 'tasteful,'

target_word: '3".' model: Doc2Vec<dm/m,d100,n5,w10,mc2,t8> similar words:
    1. 0.61 'Gigli.'
    2. 0.60 'Versus.'
    3. 0.60 'Lucy.'
    4. 0.60 '2".'
    5. 0.59 'Summer.'
    6. 0.59 'Out.'
    7. 0.58 'why.'
    8. 0.58 'Scream.'
    9. 0.58 'Water.'
    10. 0.58 'general!'

target_word: '3".' model: Doc2Vec<dm/c,d100,n5,w5,mc2,t8> similar words:
    1. 0.67 '2?'
    2. 0.66 '/>5)'
    3. 0.65 'Break.'
    4. 0.65 'mercenaries.'
    5. 0.65 'Decision.'
    6. 0.65 'A.K.A'
    7. 0.64 'Day."'
    8. 0.63 'Morgan).'
    9. 0.63 'Fenton.'
    10. 0.63 'of?'



Do the DBOW words look meaningless? That's because the gensim DBOW model
doesn't train word vectors – they remain at their random initialized values –
unless you ask with the ``dbow_words=1`` initialization parameter. Concurrent
word-training slows DBOW mode significantly, and offers little improvement
(and sometimes a little worsening) of the error rate on this IMDB
sentiment-prediction task, but may be appropriate on other tasks, or if you
also need word-vectors.

Words from DM models tend to show meaningfully similar words when there are
many examples in the training data (as with 'plot' or 'actor'). (All DM modes
inherently involve word-vector training concurrent with doc-vector training.)




Are the word vectors from this dataset any good at analogies?
-------------------------------------------------------------



In [21]:
from gensim.test.utils import datapath
questions_filename = datapath('questions-words.txt')

# Note: this analysis takes many minutes
for model in word_models:
    score, sections = model.wv.evaluate_word_analogies(questions_filename)
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print(f'{model}: {float(correct*100)/(correct+incorrect):0.2f}%% correct ({correct} of {correct+incorrect}')

2024-05-07 16:50:31,566 : INFO : Evaluating word analogies for top 300000 words in the model on C:\Users\voliveira\Documents\00-Proyectos\MyZone\.venv\Lib\site-packages\gensim\test\test_data\questions-words.txt
2024-05-07 16:50:35,290 : INFO : capital-common-countries: 0.0% (0/420)
2024-05-07 16:50:42,731 : INFO : capital-world: 0.0% (0/902)
2024-05-07 16:50:43,459 : INFO : currency: 0.0% (0/86)
2024-05-07 16:50:55,933 : INFO : city-in-state: 0.0% (0/1510)
2024-05-07 16:51:00,098 : INFO : family: 0.0% (0/506)
2024-05-07 16:51:08,359 : INFO : gram1-adjective-to-adverb: 0.0% (0/992)
2024-05-07 16:51:14,652 : INFO : gram2-opposite: 0.0% (0/756)
2024-05-07 16:51:25,577 : INFO : gram3-comparative: 0.0% (0/1332)
2024-05-07 16:51:34,260 : INFO : gram4-superlative: 0.0% (0/1056)
2024-05-07 16:51:42,543 : INFO : gram5-present-participle: 0.0% (0/992)
2024-05-07 16:51:54,576 : INFO : gram6-nationality-adjective: 0.0% (0/1445)
2024-05-07 16:52:07,366 : INFO : gram7-past-tense: 0.0% (0/1560)
2024-

Doc2Vec<dbow,d100,n5,mc2,t8>: 0.00%% correct (0 of 13617


2024-05-07 16:52:28,020 : INFO : capital-common-countries: 2.4% (10/420)
2024-05-07 16:52:35,664 : INFO : capital-world: 0.3% (3/902)
2024-05-07 16:52:36,401 : INFO : currency: 0.0% (0/86)
2024-05-07 16:52:49,497 : INFO : city-in-state: 0.1% (2/1510)
2024-05-07 16:52:53,944 : INFO : family: 39.1% (198/506)
2024-05-07 16:53:02,034 : INFO : gram1-adjective-to-adverb: 3.8% (38/992)
2024-05-07 16:53:08,332 : INFO : gram2-opposite: 6.6% (50/756)
2024-05-07 16:53:20,133 : INFO : gram3-comparative: 46.8% (623/1332)
2024-05-07 16:53:29,491 : INFO : gram4-superlative: 25.8% (272/1056)
2024-05-07 16:53:38,009 : INFO : gram5-present-participle: 24.4% (242/992)
2024-05-07 16:53:50,221 : INFO : gram6-nationality-adjective: 2.6% (37/1445)
2024-05-07 16:54:03,688 : INFO : gram7-past-tense: 28.5% (445/1560)
2024-05-07 16:54:13,969 : INFO : gram8-plural: 20.3% (242/1190)
2024-05-07 16:54:21,055 : INFO : gram9-plural-verbs: 43.2% (376/870)
2024-05-07 16:54:21,055 : INFO : Quadruplets with out-of-vocabul

Doc2Vec<dm/m,d100,n5,w10,mc2,t8>: 18.64%% correct (2538 of 13617


2024-05-07 16:54:24,546 : INFO : capital-common-countries: 2.6% (11/420)
2024-05-07 16:54:31,841 : INFO : capital-world: 0.7% (6/902)
2024-05-07 16:54:32,555 : INFO : currency: 0.0% (0/86)
2024-05-07 16:54:44,180 : INFO : city-in-state: 0.1% (1/1510)
2024-05-07 16:54:48,378 : INFO : family: 36.0% (182/506)
2024-05-07 16:54:56,471 : INFO : gram1-adjective-to-adverb: 6.4% (63/992)
2024-05-07 16:55:02,529 : INFO : gram2-opposite: 4.8% (36/756)
2024-05-07 16:55:13,425 : INFO : gram3-comparative: 35.9% (478/1332)
2024-05-07 16:55:21,766 : INFO : gram4-superlative: 27.9% (295/1056)
2024-05-07 16:55:29,706 : INFO : gram5-present-participle: 35.6% (353/992)
2024-05-07 16:55:40,862 : INFO : gram6-nationality-adjective: 1.2% (17/1445)
2024-05-07 16:55:53,177 : INFO : gram7-past-tense: 26.7% (416/1560)
2024-05-07 16:56:03,053 : INFO : gram8-plural: 11.8% (141/1190)
2024-05-07 16:56:10,027 : INFO : gram9-plural-verbs: 49.5% (431/870)
2024-05-07 16:56:10,027 : INFO : Quadruplets with out-of-vocabul

Doc2Vec<dm/c,d100,n5,w5,mc2,t8>: 17.85%% correct (2430 of 13617


Even though this is a tiny, domain-specific dataset, it shows some meager
capability on the general word analogies – at least for the DM/mean and
DM/concat models which actually train word vectors. (The untrained
random-initialized words of the DBOW model of course fail miserably.)


